In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add 'utils' folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# function to compute trajectory from dFdt
from ipynb.fs.defs.integration_dFdt import integration_dFdt

In [1]:
def gradient_flowmap(time, x, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, aux_grid, verbose = False):
    '''
    Calculates the gradient of the flowmap for a flow given by u, v, w velocities, starting from given initial conditions. 
    The initial conditions can be specified as an array. 
    
    Parameters:
        time: array (Nt,),  time array  
        x: array (3, Npoints),  array of ICs
        X: array (NY, NX, NZ)  X-meshgrid
        Y: array (NY, NX, NZ)  Y-meshgrid 
        Z: array (NY, NX, NZ)  Z-meshgrid
        Interpolant_u: Interpolant object for u(x, t)
        Interpolant_v: Interpolant object for v(x, t)
        Interpolant_w: Interpolant object for w(x, t)
        periodic: list of 4 bools, periodic[i] is True if the flow is periodic in the ith coordinate. Time is i=4.
        bool_unsteady:  specifies if velocity field is unsteady/steady
        aux_grid: array(3,), grid spacing for the auxiliary grid 
        verbose: if True plot percentage of integration done
        
    Returns:
        gradFmap: array(Nt, 3, 3, Npoints), gradient of the flowmap (3 by 3 matrix) for each time instant and each spatial point
    '''
    

    # define auxiliary grid spacing
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    rho_z = aux_grid[2]
    
    XL, XR, XU, XD, XF, XB = [], [], [], [], [], []
    
    for i in range(x.shape[1]):
        
        xr = x[0, i] + rho_x # float
        xl = x[0, i] - rho_x # float
        yu = x[1, i] + rho_y # float
        yd = x[1, i] - rho_y # float
        zF = x[2, i] + rho_z # float
        zB = x[2, i] - rho_z # float
    
        XL.append([xl, x[1, i], x[2, i]])
        XR.append([xr, x[1, i], x[2, i]])
        XU.append([x[0, i], yu, x[2, i]])
        XD.append([x[0, i], yd, x[2, i]])
        XF.append([x[0, i], x[1, i], zF])
        XB.append([x[0, i], x[1, i], zB])
    
    XL = np.array(XL).transpose()
    XR = np.array(XR).transpose()
    XU = np.array(XU).transpose()
    XD = np.array(XD).transpose()
    XF = np.array(XF).transpose()
    XB = np.array(XB).transpose()
    
    # launch trajectories from auxiliary grid
    XLend = integration_dFdt(time, XL, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, verbose)[0] # array (Nt, 2)
    XRend = integration_dFdt(time, XR, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, verbose)[0] # array (Nt, 2)
    XDend = integration_dFdt(time, XD, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, verbose)[0] # array (Nt, 2)
    XUend = integration_dFdt(time, XU, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, verbose)[0] # array (Nt, 2)
    XFend = integration_dFdt(time, XF, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, verbose)[0] # array (Nt, 2)
    XBend = integration_dFdt(time, XB, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, verbose)[0] # array (Nt, 2)
    
    return iterate_gradient(XRend, XLend, XUend, XDend, XFend, XBend)

In [2]:
from numba import njit, prange

@njit(parallel = True)
def iterate_gradient(XRend, XLend, XUend, XDend, XFend, XBend):
    
    gradFmap = np.zeros((XLend.shape[0], 3, 3, XLend.shape[2])) # array (Nt, 3, 3, Nx*Ny)
    
    for i in prange(XLend.shape[2]):      
            
        for j in prange(XLend.shape[0]):

            gradFmap[j,0,0,i] = (XRend[j,0,i]-XLend[j,0,i])/(XRend[0,0,i]-XLend[0,0,i])
            gradFmap[j,1,0,i] = (XRend[j,1,i]-XLend[j,1,i])/(XRend[0,0,i]-XLend[0,0,i])
            gradFmap[j,2,0,i] = (XRend[j,2,i]-XLend[j,2,i])/(XRend[0,0,i]-XLend[0,0,i])
        
            gradFmap[j,0,1,i] = (XUend[j,0,i]-XDend[j,0,i])/(XUend[0,1,i]-XDend[0,1,i])
            gradFmap[j,1,1,i] = (XUend[j,1,i]-XDend[j,1,i])/(XUend[0,1,i]-XDend[0,1,i])
            gradFmap[j,2,1,i] = (XUend[j,2,i]-XDend[j,2,i])/(XUend[0,1,i]-XDend[0,1,i])
                
            gradFmap[j,0,2,i] = (XFend[j,0,i]-XBend[j,0,i])/(XFend[0,2,i]-XBend[0,2,i])
            gradFmap[j,1,2,i] = (XFend[j,1,i]-XBend[j,1,i])/(XFend[0,2,i]-XBend[0,2,i])
            gradFmap[j,2,2,i] = (XFend[j,2,i]-XBend[j,2,i])/(XFend[0,2,i]-XBend[0,2,i])
            
    return gradFmap